# FF5 Stock Factors

In [ ]:
import dai
import pandas as pd

In [ ]:
sd = '2023-01-01'
ed = '2024-01-01'

## 1. FF5 Stock Factors SQL

In [ ]:
def get_ff5_sql(sql_base):
    sql = f"""--sql
    WITH
    data1 AS (
        {sql_base}
    ),

    data_mkt AS (
        SELECT DISTINCT
            date,
            c_sum(float_market_cap * change_ratio) / c_sum(float_market_cap) AS MKT
        FROM data1
    ),

    hv AS (
        WITH hv0 AS (
            SELECT
                date,
                instrument,
                change_ratio,
                float_market_cap,
                c_pct_rank(float_market_cap) AS rank_sb,
                c_pct_rank(bp_ratio)         AS rank_lmh,
                CASE WHEN rank_sb < 0.5 THEN 1 ELSE 2 END AS group_sb,
                CASE
                    WHEN rank_lmh < 0.3 THEN 1
                    WHEN rank_lmh > 0.7 THEN 3
                    ELSE 2
                END AS group_lmh
            FROM data1
        ),
        SL AS (
            SELECT DISTINCT date, c_sum(float_market_cap*change_ratio)/c_sum(float_market_cap) AS SL
            FROM hv0 WHERE group_sb=1 AND group_lmh=1
        ),
        SM_v AS (
            SELECT DISTINCT date, c_sum(float_market_cap*change_ratio)/c_sum(float_market_cap) AS SM_v
            FROM hv0 WHERE group_sb=1 AND group_lmh=2
        ),
        SH AS (
            SELECT DISTINCT date, c_sum(float_market_cap*change_ratio)/c_sum(float_market_cap) AS SH
            FROM hv0 WHERE group_sb=1 AND group_lmh=3
        ),
        BL AS (
            SELECT DISTINCT date, c_sum(float_market_cap*change_ratio)/c_sum(float_market_cap) AS BL
            FROM hv0 WHERE group_sb=2 AND group_lmh=1
        ),
        BM_v AS (
            SELECT DISTINCT date, c_sum(float_market_cap*change_ratio)/c_sum(float_market_cap) AS BM_v
            FROM hv0 WHERE group_sb=2 AND group_lmh=2
        ),
        BH AS (
            SELECT DISTINCT date, c_sum(float_market_cap*change_ratio)/c_sum(float_market_cap) AS BH
            FROM hv0 WHERE group_sb=2 AND group_lmh=3
        )
        SELECT
            SL.date AS date,
            (1/3)*(SL + SM_v + SH) - (1/3)*(BL + BM_v + BH) AS SMB_HML,
            (1/2)*(SH + BH)        - (1/2)*(SL + BL)        AS HML
        FROM SL
        JOIN SM_v USING(date)
        JOIN SH   USING(date)
        JOIN BL   USING(date)
        JOIN BM_v USING(date)
        JOIN BH   USING(date)
    ),
    hp AS (
        WITH hp0 AS (
            SELECT
                date,
                instrument,
                change_ratio,
                float_market_cap,
                c_pct_rank(float_market_cap) AS rank_sb,
                c_pct_rank(prof_ratio)       AS rank_wmr,
                CASE WHEN rank_sb < 0.5 THEN 1 ELSE 2 END AS group_sb,
                CASE
                    WHEN rank_wmr < 0.3 THEN 1
                    WHEN rank_wmr > 0.7 THEN 3
                    ELSE 2
                END AS group_wmr
            FROM data1
            QUALIFY prof_ratio IS NOT NULL
        ),
        SW AS (
            SELECT DISTINCT date, c_sum(float_market_cap*change_ratio)/c_sum(float_market_cap) AS SW
            FROM hp0 WHERE group_sb=1 AND group_wmr=1
        ),
        SM_p AS (
            SELECT DISTINCT date, c_sum(float_market_cap*change_ratio)/c_sum(float_market_cap) AS SM_p
            FROM hp0 WHERE group_sb=1 AND group_wmr=2
        ),
        SR AS (
            SELECT DISTINCT date, c_sum(float_market_cap*change_ratio)/c_sum(float_market_cap) AS SR
            FROM hp0 WHERE group_sb=1 AND group_wmr=3
        ),
        BW AS (
            SELECT DISTINCT date, c_sum(float_market_cap*change_ratio)/c_sum(float_market_cap) AS BW
            FROM hp0 WHERE group_sb=2 AND group_wmr=1
        ),
        BM_p AS (
            SELECT DISTINCT date, c_sum(float_market_cap*change_ratio)/c_sum(float_market_cap) AS BM_p
            FROM hp0 WHERE group_sb=2 AND group_wmr=2
        ),
        BR AS (
            SELECT DISTINCT date, c_sum(float_market_cap*change_ratio)/c_sum(float_market_cap) AS BR
            FROM hp0 WHERE group_sb=2 AND group_wmr=3
        )
        SELECT
            SW.date AS date,
            (1/3)*(SW + SM_p + SR) - (1/3)*(BW + BM_p + BR) AS SMB_RMW,
            (1/2)*(SR + BR)        - (1/2)*(SW + BW)        AS RMW
        FROM SW
        JOIN SM_p USING(date)
        JOIN SR   USING(date)
        JOIN BW   USING(date)
        JOIN BM_p USING(date)
        JOIN BR   USING(date)
    ),
    hi AS (
        WITH hi0 AS (
            SELECT
                date,
                instrument,
                change_ratio,
                float_market_cap,
                c_pct_rank(float_market_cap) AS rank_sb,
                c_pct_rank(inv_ratio)        AS rank_cma,
                CASE WHEN rank_sb < 0.5 THEN 1 ELSE 2 END AS group_sb,
                CASE
                    WHEN rank_cma < 0.3 THEN 1
                    WHEN rank_cma > 0.7 THEN 3
                    ELSE 2
                END AS group_cma
            FROM data1
            QUALIFY inv_ratio IS NOT NULL
        ),
        SC AS (
            SELECT DISTINCT date, c_sum(float_market_cap*change_ratio)/c_sum(float_market_cap) AS SC
            FROM hi0 WHERE group_sb=1 AND group_cma=1
        ),
        SM_i AS (
            SELECT DISTINCT date, c_sum(float_market_cap*change_ratio)/c_sum(float_market_cap) AS SM_i
            FROM hi0 WHERE group_sb=1 AND group_cma=2
        ),
        SA AS (
            SELECT DISTINCT date, c_sum(float_market_cap*change_ratio)/c_sum(float_market_cap) AS SA
            FROM hi0 WHERE group_sb=1 AND group_cma=3
        ),
        BC AS (
            SELECT DISTINCT date, c_sum(float_market_cap*change_ratio)/c_sum(float_market_cap) AS BC
            FROM hi0 WHERE group_sb=2 AND group_cma=1
        ),
        BM_i AS (
            SELECT DISTINCT date, c_sum(float_market_cap*change_ratio)/c_sum(float_market_cap) AS BM_i
            FROM hi0 WHERE group_sb=2 AND group_cma=2
        ),
        BA AS (
            SELECT DISTINCT date, c_sum(float_market_cap*change_ratio)/c_sum(float_market_cap) AS BA
            FROM hi0 WHERE group_sb=2 AND group_cma=3
        )
        SELECT
            SC.date AS date,
            (1/3)*(SC + SM_i + SA) - (1/3)*(BC + BM_i + BA) AS SMB_CMA,
            (1/2)*(SC + BC)        - (1/2)*(SA + BA)        AS CMA
        FROM SC
        JOIN SM_i USING(date)
        JOIN SA   USING(date)
        JOIN BC   USING(date)
        JOIN BM_i USING(date)
        JOIN BA   USING(date)
    ),

    data_ff5 AS (
        SELECT
            m.date,
            m.MKT,
            (hv.SMB_HML + hp.SMB_RMW + hi.SMB_CMA) / 3 AS SMB,
            hv.HML,
            hp.RMW,
            hi.CMA
        FROM data_mkt m
        JOIN hv USING(date)
        JOIN hp USING(date)
        JOIN hi USING(date)
        QUALIFY COLUMNS(*) IS NOT NULL
    )
    SELECT *
    FROM data_ff5
    ORDER BY date
    """
    return sql


## 2. FF5 Factors

### 2.1 FF5 Daily

In [ ]:
sql_ff5_base_daily = f"""--sql
SELECT
    date,
    instrument,
    change_ratio,
    float_market_cap,
    1 / pb                     AS bp_ratio,
    net_profit_rate_lf         AS prof_ratio,
    invested_capital_lf_yoy    AS inv_ratio
FROM cn_stock_prefactors
WHERE instrument NOT LIKE '%BJ%'
"""

sql_ff5_daily = get_ff5_sql(sql_ff5_base_daily)

In [ ]:
df_ff5_daily = dai.query(sql_ff5_daily, filters={"date":[sd, ed]}).df()
df_ff5_daily

### 2.2 FF5 Weekly